In [1]:
import pandas as pd
from metasynth import MetaDataset
import wget
from pathlib import Path
import json

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}

In [3]:
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

In [4]:
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [5]:
dataset = MetaDataset.from_dataframe(df)

In [6]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,<NA>,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,<NA>,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,<NA>,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,<NA>,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,<NA>,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
dataset.synthesize(1000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,2,"o R'LzyJ91p x]V BVMX;,GjaN<hB0[3 Yzs<jt:&0Q x$...",male,10.287405,1,0,J-_m8,4.298321,<NA>,S
1,2,0,1,"UQ;4jUbb4+:h=-;EhM.H\|G4>7N0//?6XGzMJJ+`,hlj>]...",female,50.504409,0,2,3,27.737566,<NA>,S
2,3,0,3,)(Eo{ sJ 7Cv_`Df,female,39.537772,2,0,21,21.680570,<NA>,S
3,4,1,1,"v YXb-vgRJq9tn+'J{F '{4i6;7wi'n~u3""]@zh<~zc01...",male,35.847946,2,2,454,39.461221,<NA>,S
4,5,1,3,"m;!""# @]^>RtaD 7Tn!P?x=q|q+P(NOAV6v&_/JZ ""=w'""...",female,NaN,0,1,8664000,67.575527,<NA>,C
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,1,1,"kgX(7""#@rvpC_:m+ /U6I{""9};B\mX"" j",male,NaN,0,0,xJV7xcu8446,84.150495,<NA>,S
996,997,1,1,"ANKQ/4_}3SI r{uS.VO%G o(m>;PguCmi|K""wn4 ] ;|gz...",male,32.162074,0,0,9438,31.768021,<NA>,S
997,998,1,1,"}2] Tnd""M[K /P6Mc ?CMebkL-bpL:`Sw}z4i=:HO\!tj$...",male,6.901781,0,0,98885,33.620306,<NA>,C
998,999,0,3,{nM])vyA`qL s`=SpSZ[4lEzLdSS6](6@sPU> n?.z y4|...,male,34.992752,0,0,008137,0.303461,<NA>,S


In [8]:
dataset.to_json("test.json", validate=True)

In [9]:
print(MetaDataset.from_json("test.json"))

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'type': 'IntVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'UniqueKeyDistribution', 'parameters': {'low': 1, 'consecutive': 1}}"}

{'name': 'Survived', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['0', '1'], 'counts': [549, 342]}}"}

{'name': 'Pclass', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['1', '2', '3'], 'counts': [216, 184, 491]}}"}

{'name': 'Name', 'type': 'StringVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,81}[)][ ]'}

{'name': 'Sex', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['female', 'male'], 'counts': [314, 577]}}"}

{'name': 'Age', 'type': 'FloatVar', 'dtype': 'float64

In [10]:
dataset.to_dict()

{'n_rows': 891,
 'n_columns': 12,
 'vars': [{'name': 'PassengerId',
   'type': 'IntVar',
   'dtype': 'int64',
   'prop_missing': 0.0,
   'distribution': {'name': 'UniqueKeyDistribution',
    'parameters': {'low': 1, 'consecutive': 1}}},
  {'name': 'Survived',
   'type': 'CategoricalVar',
   'dtype': 'category',
   'prop_missing': 0.0,
   'distribution': {'name': 'CatFreqDistribution',
    'parameters': {'categories': array(['0', '1'], dtype='<U1'),
     'counts': array([549, 342])}}},
  {'name': 'Pclass',
   'type': 'CategoricalVar',
   'dtype': 'category',
   'prop_missing': 0.0,
   'distribution': {'name': 'CatFreqDistribution',
    'parameters': {'categories': array(['1', '2', '3'], dtype='<U1'),
     'counts': array([216, 184, 491])}}},
  {'name': 'Name',
   'type': 'StringVar',
   'dtype': 'string',
   'prop_missing': 0.0,
   'distribution': {'name': 'RegexDistribution',
    'parameters': {'re_list': [('.[]{12,81}', 1.0),
      ('[)]', 0.16049382716049382),
      ('[ ]', 0.0011223

In [11]:
print(MetaDataset.from_xml("test.xml"))

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
dataset.to_dict()